<a href="https://colab.research.google.com/github/yuliiabosher/Cyber_Resilience_Course/blob/main/CDProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install googlemaps

In [ ]:
import googlemaps
import os
import requests
import json
from google.colab import output

In [ ]:
google_key = input("Enter the Google maps API key: ")
os.environ['GOOGLE_API_KEY'] = google_key
output.clear()

In [ ]:
gmaps = googlemaps.Client(os.environ.get('GOOGLE_API_KEY'))

def get_schools(lat,lon,rad):
  school_list=[]
  try:
    places_result = gmaps.places_nearby(location= f"{lat},{lon}", radius = rad, type = "school")
    for place in places_result['results']:
      school_name= place['name']
      school_location_lat= place['geometry']['location']['lat']
      school_location_lng= place['geometry']['location']['lng']
      print(f"{school_name}, {school_location_lat},{school_location_lng}")
      school_list.append([school_name, school_location_lat,school_location_lng])
    return school_list
  except Exception as e:
    print("Error: " + str(e))
result = get_schools(54.966667, -1.600000,20000)

Harton Academy, 54.97601519999999,-1.4064828
Jesmond Park Academy, 54.9939443,-1.588097
make a wish, 54.9762162,-1.6383755
Richard Coates C Of E Primary School, 55.0523247,-1.7459373
Saint James Roman Catholic V.A. Primary School, 54.967381,-1.5089456
Gateshead College, 54.96657199999999,-1.5981311
Jesmond Pool & Gym, 54.99409919999999,-1.6061812
Royal Grammar School Newcastle, 54.9831764,-1.6073674
Excelsior Academy, 54.9748828,-1.6905524
St Aidan's Catholic Academy, 54.89307749999999,-1.3883967
Dame Allan's Schools, 54.98253940000001,-1.6524177
Riverside Primary Academy, 54.9564501,-1.6441569
Cramlington Learning Village, 55.07585989999999,-1.5933104
John Spence Community High School, 55.02233369999999,-1.4514443
Barnes Infant Academy, 54.89707540000001,-1.4094252
Ebchester C of E Primary School, 54.89326860000001,-1.8392587
St. Bede's Catholic School & Sixth Form College, 54.8283727,-1.7477169
North Gosforth Academy, 55.05801739999999,-1.6193522
Stephenson Memorial Primary School, 5

In [ ]:
aws_key = input("Enter the AWS API key: ")
os.environ['AWS_API_KEY'] = aws_key
output.clear()

In [43]:
def add_schools():
  url = "https://a595c6ya39.execute-api.eu-west-2.amazonaws.com/default/myPythonProjectFunc"

  payload = json.dumps({
    "data": result
  })
  headers = {
    'Content-Type': 'application/json',
    'x-api-key': os.environ.get('AWS_API_KEY')
  }

  response = requests.request("POST", url, headers=headers, data=payload)

  print(response.text)
add_schools()

{"message": "These schools are already in a file", "data": []}


In [44]:
def show_schools():
  url = "https://a595c6ya39.execute-api.eu-west-2.amazonaws.com/default/myPythonProjectFunc"
  payload = ""
  headers = {
    'x-api-key': os.environ.get('AWS_API_KEY')
  }

  response = requests.request("GET", url, headers=headers, data=payload)

  print(response.text)
show_schools()

{"message": "The data has been found", "data": [["school1", " 123", " 526"], ["school2", " 124", " 586"], ["school3", " 789", " 524"], ["school10", "8", "9"], ["school4", "1", "2"], ["school5", "3", "4"], ["school100", "1", "2"], ["school22", "1", "2"], ["school222", "1", "2"], ["school55", "1", "2"], ["school66", "3", "4"], ["Bannockburn Primary School & Nursery", "51.4869172", "0.1015561"], ["St Margaret Clitherow Primary School", "51.50103289999999", "0.1132992"], ["Parkway Primary School", "51.49280040000001", "0.1341053"], ["St Michael's East Wickham C of E Primary School", "51.46934", "0.119751"], ["Jo Richardson Community School", "51.53407139999999", "0.1262779"], ["Woolwich Polytechnic School for Boys", "51.5037284", "0.1081882"], ["Eastbury Community School", "51.5415693", "0.09116149999999999"], ["King Henry School", "51.47343300000001", "0.166117"], ["Sydney Russell School", "51.5486205", "0.1331252"], ["Langdon Academy", "51.53266139999999", "0.0684394"], ["The Garard Acad